In [1]:
import numpy as np
import math
import edward as ed
import tensorflow as tf
import plotly.plotly as py
import plotly.graph_objs as go
import seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import GPy
GPy.plotting.change_plotting_library('plotly')
from IPython.display import display
init_notebook_mode(connected=True)
from scipy.special import expit

In [2]:
N = 500
D = 2

X = np.sort(np.random.uniform(0, 1, (N, D)))
k = GPy.kern.RBF(input_dim=D,lengthscale=0.2)
mu = np.ones((N))*2 # vector of means
C = k.K(X, X) # covariance matrix
rate = np.exp(np.random.multivariate_normal(mu,C)) 

In [3]:
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z=rate, mode = 'markers', marker=dict(
        size = 2,
    ))])

In [ ]:
counts = np.random.poisson(abs(rate + np.random.normal(0, 5, size = (N,))))
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z=counts, mode = 'markers', marker=dict(size = 2,))])

## Edward

In [ ]:
from edward.models import MultivariateNormalTriL, Normal, Poisson
from edward.util import rbf

In [ ]:
x_ph = tf.placeholder(tf.float32, (N,D))  # inputs to Gaussian Process
K = rbf(x_ph, lengthscale=0.2) + tf.diag(tf.ones(N)*1e-3)
f = MultivariateNormalTriL(loc=tf.ones((N))*5, scale_tril=tf.cholesky(K))
y = Poisson(rate=tf.exp(f))

In [ ]:
qf = Normal(loc=tf.Variable(tf.random_normal([N])),
            scale=tf.nn.softplus(tf.Variable(tf.random_normal([N]))))

inference = ed.KLqp({f: qf}, data={x_ph: X, y: counts})
inference.run(n_iter=5000)
pred = np.exp(qf.mean().eval())

In [ ]:
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z=pred, mode = 'markers', marker=dict(
        size = 2,
    ))])